In [152]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [153]:
import astropy.io.fits as pf

In [154]:
import pyqtgraph as pq

In [155]:
run /Users/aaronresnick/Desktop/CODERepository/pteropractice/CHZPHOTNINOX.py

In [156]:
run /Users/aaronresnick/Desktop/CODERepository/stats.py

#load in selected darks
dlist = ((28,112,144,171,248,354,393,468,585,704,806))
dks = zeros((11,512,640))
for i in range(len(dlist)):
    num = str(dlist[i])
    dks[i,:,:] = pf.getdata('/Users/aaronresnick/Desktop/Ninox/Ninox/NinoxOccultation'+str(num.zfill(4))+'.fits')

#load in flats
flats = zeros((150,512,640))
for i in range(len(flats)):
    num = str(i)
    flats[i,:,:] = pf.getdata('/Users/aaronresnick/Desktop/Ninox/Ninox/NinoxFlatOneSecond'+str(num.zfill(3))+'.fits')

#load in flats
rawd = zeros((850,512,640))
for i in range(len(rawd)):
    num = str(i)
    rawd[i,:,:] = pf.getdata('/Users/aaronresnick/Desktop/Ninox/Ninox/NinoxOccultation'+str(num.zfill(4))+'.fits')

rawd = rawd[:,40:,:590]
flats = flats[:,40:,:590]
dks = dks[:,40:,:590]

#compare = median(flats[90:150,:,:],axis = 0) - median(flats[20:80,:,:],axis = 0)
#print(mean(compare,axis = 0))

masterflat = mean(flats[20:80,:,:],3,axis = 0) - mean(flats[90:150,:,:],3,axis = 0)

masterflatb = copy(masterflat)

masterflatb.shape

masterflatb = masterflatb[1:471,1:589]

masterflatb.shape

masterflatb

masterflatb = 1.0*masterflatb/(1.0*mean(masterflatb,3))

pf.writeto('masterflatb.fits',masterflatb,clobber=True)
masterflatb

meds = []
for i in range(len(dks)):
    meds.append(median(dks[i,:,:]))
    dks[i,:,:] = dks[i,:,:] - median(dks[i,:,:])
dks = median(dks,axis = 0) + median(meds)      
pf.writeto('dks.fits',dks,clobber=True)

data = 1.0*rawd - 1.0*dks
#data = 1.0*data/(1.0*masterflat)
data = data[600:,:,:]
#for i in range(len(data)):
#    for j in range(len(data[1,1,:])):
#         print(median(data[i,:,j]))

dkmn,dkstd,rmn,rstd,dmn,dstd

In [157]:
#for i in range(len(data)):
#    for j in range(len(data[1,1,:])):
#        if 1.0*mean(data[i,:,j],5) == 0:
#            print('zero')
#        data[i,:,j] = (1.0*data[i,:,j])/(1.0*mean(data[i,:,j],5))

In [158]:
#pf.writeto('data_w_flat.fits',data,clobber=True)

In [159]:
#flats = median(flats,axis = 0)

In [160]:
#flats = (1.0*flats)/(1.0*median(flats))

In [161]:
#rawd = rawd - dks
#data = rawd[600:,:,:]

In [162]:
#pf.writeto('data.fits',rawd)

In [163]:
data = pf.getdata('data_w_flat.fits')

In [164]:
#rawd = (1.0*rawd)/(1.0*flats)

#Geometry

In [165]:
occstar = data[:,245:305,305:345]
compa = data[:,50:88,518:561]
compb = data[:,74:134,269:329]
compc = data[:,280:315,90:140]
compd = data[:,100:140,40:80]

In [166]:
#Hot pixel shit
ccdata = (copy(data)**2)
dlist = ((occstar,compa,compb,compc,compd))
xStorage = zeros((len(dlist),len(data)))
yStorage = zeros((len(dlist),len(data)))
for h in range(len(dlist)): 
    for i in range(len(dlist[h])):
        hotpix = dlist[h][i,:,:].argmax()
        x = hotpix % dlist[h][1,1,:].shape
        y = hotpix // dlist[h][1,1,:].shape
        if h == 1:
            xStorage[h,i] = x + 518
            yStorage[h,i] = y + 50
            #compa[i,y,x] = 2000000
            #ccdata[i,yStorage[h,i]-5:yStorage[h,i]+5,xStorage[h,i]-5:xStorage[h,i]+5] = 100000 #-> use to check if hotpix = center of stars
        if h == 2:
            xStorage[h,i] = x + 269
            yStorage[h,i] = y + 74
            #ccdata[i,yStorage[h,i],xStorage[h,i]] = 5000#ccdata[i,yStorage[h,i]-0:yStorage[h,i]+0,xStorage[h,i]-0:xStorage[h,i]+0] = 100000 #-> use to check if hotpix = center of stars
        if h == 3:
            xStorage[h,i] = x + 90
            yStorage[h,i] = y + 280
            #ccdata[i,yStorage[h,i],xStorage[h,i]] = 5000**2#-0:yStorage[h,i]+0,xStorage[h,i]-0:xStorage[h,i]+0] = 100000 #-> use to check if hotpix = center of stars
        if h == 4:
            xStorage[h,i] = x + 40
            yStorage[h,i] = y + 100
            #ccdata[i,yStorage[h,i],xStorage[h,i]] = 5000#-0:yStorage[h,i]+0,xStorage[h,i]-0:xStorage[h,i]+0] = 100000 #-> use to check if hotpix = center of stars

In [167]:
#clf()
#pq.show(ccdata[:,100:140,40:80])

In [168]:
def banana(denom,axis,std):
    global yStorage
    global xStorage
    xmn,xstd = robomad(xStorage[axis],std)
    ymn,ystd = robomad(yStorage[axis],std)

    for i in range(1,len(xStorage[axis])-1):
        if xStorage[axis,i] > xStorage[axis,i-1]+(xstd/denom) or xStorage[axis,i] < (xStorage[axis,i-1]-(xstd/denom)):
            xStorage[axis,i] = ((xStorage[axis,i-1]+xStorage[axis,i+1])/2)
        if yStorage[axis,i] > yStorage[axis,i-1]+(ystd/denom) or yStorage[axis,i] < yStorage[axis,i-1]-(xstd/denom):
            yStorage[axis,i] = ((yStorage[axis,i-1]+yStorage[axis,i+1])/2)

In [169]:
def rutabaga(ccdata,r1,r2,r3,axis):
    global xStorage
    global yStorage
    
    for j in range(4):
        #print(j)
        for i in range(len(ccdata[:,:,:])):
            x,y = cntrd(ccdata[i,:,:],r1,xStorage[axis,i],yStorage[axis,i])
            #ccdata[i,y,x] = 2500
            xStorage[axis,i] = x
            yStorage[axis,i] = y
            #print i
    for j in range(4):
        #print(j)
        for i in range(len(ccdata[:,:,:])):
            x,y = cntrd(ccdata[i,:,:],r2,xStorage[axis,i],yStorage[axis,i])
            #ccdata[i,y,x] = 2500
            xStorage[axis,i] = x
            yStorage[axis,i] = y
            #print i
    for j in range(4):
        #print(j)
        for i in range(len(ccdata[:,:,:])):
            x,y = cntrd(ccdata[i,:,:],r3,xStorage[axis,i],yStorage[axis,i])
            #ccdata[i,y,x] = 2500
            xStorage[axis,i] = x
            yStorage[axis,i] = y
            #print i
    for i in range(len(ccdata[:,:,:])):
        x,y = cntrd(ccdata[i,:,:],r3,xStorage[axis,i],yStorage[axis,i])
        #ccdata[i,y,x] = 100000
        xStorage[axis,i] = x
        yStorage[axis,i] = y
        #print i

In [170]:
def pineapple(denom,axis,std): #IF THE PROBLEM COMES LATER IN THE DATASET
    global yStorage
    global xStorage
    xmn,xstd = robomad(xStorage[axis],std)
    ymn,ystd = robomad(yStorage[axis],std)

    for i in range(len(xStorage[axis])):
        if xStorage[axis,i] > xStorage[axis,i-1]+(xstd/denom) or xStorage[axis,i] < (xStorage[axis,i-1]-(xstd/denom)):
            xStorage[axis,i] = xStorage[axis,i-1]
        if yStorage[axis,i] > yStorage[axis,i-1]+(ystd/denom) or yStorage[axis,i] < yStorage[axis,i-1]-(xstd/denom):
            yStorage[axis,i] = (yStorage[axis,i-1])

In [171]:
def mangosteen(denom,axis,std): #IF THE PROBLEM COMES EARLIER IN THE DATASET
    global yStorage
    global xStorage
    xmn,xstd = robomad(xStorage[axis],std)
    ymn,ystd = robomad(yStorage[axis],std)

    for i in range(len(xStorage[axis])-2,-1,-1):
        if (xStorage[axis,i] > (xStorage[axis,i+1]+(xstd/denom))) or xStorage[axis,i] < (xStorage[axis,i+1]-(xstd/denom)):                    
            xStorage[axis,i] = (xStorage[axis,i+1])
            #print('mangosteen!')
        if yStorage[axis,i] > (yStorage[axis,i+1]+(ystd/denom)) or yStorage[axis,i] < (yStorage[axis,i+1]-(xstd/denom)):
            yStorage[axis,i] = (yStorage[axis,i+1])

In [172]:
banana(5,1,3)
rutabaga(ccdata,15,10,5,1)
banana(5,1,3)
rutabaga(ccdata,6,6,6,1)
banana(5,1,3)
rutabaga(ccdata,6,6,6,1)

In [173]:
banana(5,2,3)
rutabaga(ccdata,15,10,10,2)
banana(5,2,3)
rutabaga(ccdata,10,6,6,2)
banana(5,2,3)
rutabaga(ccdata,10,6,6,2)
banana(5,2,3)
rutabaga(ccdata,10,6,6,2)
banana(5,2,3)
rutabaga(ccdata,10,6,6,2)
banana(6,2,3)
rutabaga(ccdata,10,6,6,2)
banana(6,2,3)
pineapple(2,2,3)
rutabaga(ccdata,6,6,6,2)
banana(7,2,3)
mangosteen(2,2,3)
rutabaga(ccdata,6,6,4,2)
banana(10,2,3)
rutabaga(ccdata,4,4,4,2)
banana(10,2,3)

In [174]:
banana(5,3,3)
rutabaga(ccdata,6,6,6,3)
banana(5,3,3)
rutabaga(ccdata,6,6,6,3)
banana(5,3,3)
pineapple(3,3,3)
rutabaga(ccdata,6,6,6,3)
banana(6,3,3)
mangosteen(3,3,3)
rutabaga(ccdata,6,6,6,3)
banana(6,3,3)
pineapple(3,3,2)
rutabaga(ccdata,6,6,6,3)
banana(5,3,3)
mangosteen(3,3,2)
rutabaga(ccdata,6,6,6,3)
banana(6,3,3)
rutabaga(ccdata,6,6,6,3)
banana(6,3,3)
rutabaga(ccdata,6,6,6,3)
banana(6,3,3)
rutabaga(ccdata,6,6,6,3)
banana(6,3,3)
mangosteen(3,3,2)
rutabaga(ccdata,6,6,6,3)

In [175]:
banana(5,4,3)
rutabaga(ccdata,10,10,6,4)
banana(5,4,3)
rutabaga(ccdata,8,8,8,4)
banana(5,4,3)
rutabaga(ccdata,8,6,6,4)
banana(7,4,3)
rutabaga(ccdata,8,6,6,4)
banana(7,4,3)
rutabaga(ccdata,8,6,6,4)
banana(7,4,3)

In [176]:
#for i in range(len(xStorage[1])):
#    ccdata[i,yStorage[1,i],xStorage[1,i]] = 10000**2
#for i in range(len(xStorage[1])):
#    ccdata[i,yStorage[2,i],xStorage[2,i]] = 10000**2
#for i in range(len(xStorage[1])):
#    ccdata[i,yStorage[3,i],xStorage[3,i]] = 10000**2
#for i in range(len(xStorage[1])):
#    ccdata[i,yStorage[4,i],xStorage[4,i]] = 10000**2

In [177]:
#clf()
#pq.show(ccdata[:,50:88,510:571])
#matshow(sum(data[0:10,:,:],axis = 0))

In [178]:
Pfrm1 = 5
Pxc1,Pyc1 = cntrd(sum(ccdata[5:10,:,:],axis=0),5,333,275)

In [179]:
xDist1 = Pxc1 - xStorage[1,0]
yDist1 = Pyc1 - yStorage[1,0]

In [180]:
for i in range(len(data)):
    #x,y = cntrd(data[i,:,:],4,mgx*i+bs[h][0],mgy*i+bs[h][1])
    #xd = Pmgy*i+bs[h][0]
    #yd = Pmgx*i+bs[h][1]
    xStorage[0,i] = xDist1 + xStorage[1,i]
    yStorage[0,i] = yDist1 + yStorage[1,i]
    #ccdata[i,yDist1 + yStorage[1,i], xDist1 + xStorage[1,i]] = 100000# -0:yDist1 + yStorage[0,i] +0,xDist1 + xStorage[0,i]-0:xDist1 + xStorage[0,i]+0] = 100000

In [181]:
banana(5,0,3)

In [182]:
#for i in range(len(xStorage[0])):
#    ccdata[i,yStorage[0,i],xStorage[0,i]] = 100000

In [183]:
#clf()
#pq.show(data[:,245:305,305:345])

In [184]:
#data = data [55:65,:,:]

In [185]:
# NEED BETTER NUMBERS FOR FWHM, APR, INR, OUTR
# FOR REFERENCE: dlist = ((occdata,compa,compb,compd))
dlist = ((data,data,data,data,data))#,compa,compb,compd))
fluxStorage = zeros((len(dlist),len(data)))
bgFluxStorage = zeros((len(dlist),len(data)))
apsets = [[None,8.,17.,28.],[8.,8.,17.,28.],[8.,8,14.,28.],[4,8,14.,28.],[4,8,14.,28.]]#[6.,6.,7.,14.]]
for h in range(len(dlist)):
    #print(h)
    for i in range(len(data)):
        flux,bgflux = apPhot(h,data[i,:,:],xStorage[h,i],yStorage[h,i],apsets[h][0],apsets[h][1],apsets[h][2],apsets[h][3])
        fluxStorage[h,i] = flux
        bgFluxStorage[h,i] = bgflux

In [186]:
pf.writeto('makedupdata.fits',data,clobber=True)

In [187]:
occflux = fluxStorage[0,:]
compa = fluxStorage[1,:]
compb = fluxStorage[2,:]
compc = fluxStorage[3,:]

In [188]:
master = compa+compb+compb

In [189]:
std(master[:50]),std(occflux[:50])

(7665.122638171205, 3140.0410803029645)

In [190]:
clf()
plot(occflux)
plot(compa)
#plot(compb)
#plot(compc)
#plot(compd)

In [191]:
occflux = occflux/occflux[0]
compa = compa/compa[0]
compb = compb/compb[0]
compc = compc/compc[0]
master = master/master[0]

In [192]:
clf()
plot(occflux/master)#+compb[50:])/2))

In [193]:
#clf()
#pq.show(data)
f = occflux/master

In [194]:
numpy.polyfit(range(18),f[152:170],1),numpy.polyfit(range(18),f[205:223],1)

(array([-0.04640036,  1.03349983]), array([ 0.04690023,  0.10356241]))

In [195]:
x = array(range(250))
xf = array(range(250))

In [196]:
y = (-0.04640074)*(x-152)+1.03350681
yf = (0.04690044)*(x-205)+0.10356409

In [197]:
h = ones((len(y)))

In [198]:
h = h*0.55

In [231]:
clf()
plot(y)
plot(f)
plot(h)
plot(yf)
plt.text(-10,11.5,'Plot starts at frame 600; halflight at: Ingress: x = 162.419 ; Egress: x = 214.521')
plt.text(-13,10.5,'Ninox cadence was 0.5 hz, so halflight to halflight (214.521-162.419)*2 = 104.204s')

In [232]:
savefig('halflight.png')

In [200]:
214.521-162.419

52.101999999999975

In [202]:
52.101999999999975*2

104.20399999999995